<a href="https://colab.research.google.com/github/7vvXi/writeEnglish/blob/master/LexRank_singleLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LexRankとsingleLSTMを用いた英文の生成

## kerasと分かち書き用のライブラリ等をインポート

In [1]:
!pip3 install sumy
!pip3 install keras
!pip3 install googletrans
!pip3 install nltk
!pip3 install numpy

In [2]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from googletrans import Translator
import nltk
import numpy as np
import random
import sys
import io
import os
import glob

Using TensorFlow backend.


In [3]:
!python -c "import nltk; nltk.download('punkt')"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
!python -m pip install keras --upgrade

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title ここまで実行したら再起動

## 初期設定

In [0]:
#入力用の各章のファイルが置いてあるディレクトリパス
DIR = ""
#出力用のファイル名
out_path = "docA.txt"
#LexRankを使用した際に何文取り出すか
OUTSEC = 5
#最終的な出力を何文にするか
OUTSEN = 5
#いくつの単語で1つの文とするか
maxlen = 4
#何単語ずつ変化させていくか
step = 1

In [8]:
#移動
%cd drive/My\ Drive/proX_summary/Generated\ type/data/docA/

/content/drive/My Drive/proX_summary/Generated type/data/docA


In [9]:
!ls

docA1.txt  docA3.txt  docA5.txt  docA7.txt
docA2.txt  docA4.txt  docA6.txt  docA8.txt


## 各章ごとにLexRankを用いて任意数を抜き出し、連結

In [0]:
def sectionLex():
  #言語は英語に設定
  LANG = "english"
  #.txtファイルを全て選択
  file = glob.glob('*.txt')
  ex = []
  for i in range(len(file)):
    parser = PlaintextParser.from_file(file[i], Tokenizer(LANG))
    stemmer = Stemmer(LANG)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANG)
    for sentence in summarizer(parser.document, OUTSEC):
      ex.append(str(sentence) + '\n')
    #UTF-8で出力
    with open('../../../../../../output.txt', mode='w', encoding='utf-8') as f:
      f.writelines(ex)

In [0]:
#LexRank呼び出し
sectionLex()

In [12]:
%cd ../../../../../../

/content


##分かち書きと辞書登録

In [13]:
#UTF-8でファイルを開く&小文字にする
with io.open('output.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('charcter count:', len(text))

charcter count: 9210


In [0]:
#単語ごとに分解（分かち書き）
text = nltk.word_tokenize(text)
chars = text
#正順の辞書リスト
chr_index = {}
#逆順の辞書リスト
rvs_index = {}
#文のリスト
sentences = []
#次の単語
next_word = []

In [15]:
#正順のリストを作成
count = 0
for c in chars:
    if not c in chr_index:
        chr_index[c] = count
        count += 1
        print(count, c)
#逆順のリストを作成
rvs_index = dict([(value, key) for (key, value) in chr_index.items()])

1 in
2 general
3 ,
4 iot
5 can
6 benefit
7 from
8 the
9 virtually
10 unlimited
11 capabilities
12 and
13 resources
14 of
15 cloud
16 to
17 compensate
18 its
19 technological
20 constraints
21 (
22 e.g.
23 storage
24 processing
25 communication
26 )
27 .
28 being
29 characterized
30 by
31 a
32 very
33 high
34 heterogeneity
35 devices
36 technologies
37 protocols
38 it
39 lacks
40 different
41 important
42 properties
43 such
44 as
45 scalability
46 interoperability
47 flexibility
48 reliability
49 efficiency
50 availability
51 security
52 consequence
53 analyses
54 unprecedented
55 complexity
56 are
57 possible
58 data-driven
59 decision
60 making
61 prediction
62 algorithms
63 be
64 employed
65 at
66 low
67 cost
68 providing
69 means
70 for
71 increasing
72 revenues
73 reduced
74 risks
75 speed
76 networks
77 enables
78 effective
79 monitoring
80 control
81 remote
82 things
83 their
84 coordination
85 communications
86 real-time
87 access
88 produced
89 data
90 this
91 represents
92 ano

In [16]:
for i in range(0, len(text) - maxlen, step):
    #maxlen個の単語を部分文字列(1文)として格納
    sentences.append(text[i: i + maxlen])
    #格納した部分文字列の次の単語を格納
    next_word.append(text[i + maxlen])
#部分文字列の数を出力
print('sentence count:', len(sentences))

sentence count: 1487


## 単語のベクトル化

In [0]:
#np.bool型の3次元配列：(部分文字列の数、部分文字列の最大長、単語数) -> 単語ごとの出現頻度
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
#np.bool型の2次元配列：(部分文字列の数、単語数) -> 次の単語を予測する用
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
#各部分文字列をベクトル化する
for i, sentence in enumerate(sentences):
    for t, ch in enumerate(sentence):
        x[i, t, chr_index[ch]] = 1
    y[i, chr_index[next_word[i]]] = 1

##モデルの作成

[Sequentialモデルについて](https://keras.io/ja/getting-started/sequential-model-guide/)

Sequentialモデルは、層を積み上げるだけの単純なモデル。1層前の出力をそのまま入力するイメージ。ノード間にはすべてエッジを引く。

In [18]:
#単純モデルを作る
model = Sequential()
#LSTMを使用。バッチサイズは128。
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
#単語ごとに見ていくので、単語数で出力。よさげなのでsofmax。
model.add(Dense(len(chars), activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


[RMSpropについて](https://qiita.com/tokkuman/items/1944c00415d129ca0ee9#rmsprop)

In [0]:
#勾配法にRMSpropを用いる
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

##各単語の出現率を計算し、出力する単語を選ぶ

In [0]:
# preds       ：モデルからの出力。float32型。
# temperature ：多様度。低いほど出現率の高いものが選ばれやすくなる。
#モデルから出力は多項分布の形のため、総和は1.0になる
def selectWD(preds, temperature=1.0):
    #float64型へ変換
    preds = np.asarray(preds).astype('float64')
    #確率の低い単語が選ばれやすくなるように、自然対数を多様度で割る
    preds = np.log(preds) / temperature
    #確率の自然対数を逆変換する（自然指数関数にする）
    exp_preds = np.exp(preds)
    #総和が1になるように、全値を総和で割る
    preds = exp_preds / np.sum(exp_preds)
    #多項分布に従って、ランダムに選ぶ
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## 各epochごとに呼ばれる処理

In [0]:
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    #最初の4単語を部分文字列からランダムに選ぶ場合
    #start_index = random.randint(0, len(text) - maxlen - 1)
    #入力文字列の一番最初にする
    start_index = 0
    #diversity：多様性。selectWDのtemperatureと同じ。高いほど確率が低い文字も選ばれる。
    for diversity in [0.2]:
        print('----- diversity:', diversity)
        #出力用
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += " ".join(sentence)
        print(" ".join(sentence))

        print('----- Generating with seed: "' + " ".join(sentence)+ '"')
        sys.stdout.write(generated)
        
        #OUTSEN個の文を出力、あるいは1000単語出力で終了
        flag = OUTSEN
        for i in range(1000):
            #現在の文中のどの位置に何の単語があるか
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, ch in enumerate(sentence):
                x_pred[0, t, chr_index[ch]] = 1.
            #次の単語を予測する
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = selectWD(preds, diversity)
            next_char = rvs_index[next_index]
            #最初の単語を削り、後ろに予測された単語を追加
            sentence = sentence[1:]
            sentence.append(next_char)
            #出力整理
            if next_char == '.':
                flag -= 1
                generated += next_char + "\n"
                sys.stdout.write(next_char+"\n")
            elif next_char == ',':
                generated += next_char
                sys.stdout.write(next_char)
            else:
                generated += " " + next_char
                sys.stdout.write(" "+next_char)
            sys.stdout.flush()
            if flag <= 0:
                break
        sys.stdout.flush()
        print()

In [0]:
#各エポック時に上記処理を呼ぶように設定
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## フィッティング実施

In [23]:
#バッチサイズ128、エポック数100、前記述の関数を呼び出す
model.fit(x, y, batch_size=128, epochs=100, callbacks=[print_callback])


Epoch 1/100
1487/1487 [==============================] - 2s 1ms/step - loss: 6.6007

----- Generating text after Epoch: 0
----- diversity: 0.2
in general , iot
----- Generating with seed: "in general , iot"
in general , iot,, and, and,,, and,,,,,,,,,, and the,,,, and,,,, and,,,, and,, and and,,,,,,,, and and,,,,,,,,, and and,,, and,, and,,, and, and,,,,,,,,,,,, and,, and,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, and,,,,,,,, the,, of, in,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, and, and,,,,,,,,,,,,, and,, and,,,,,,,,,,,,,,,,,,,,,,,,,,,, and,,,,, and,,,,,,, and,,,,,,,,,,,,, and,,,,,,,, and,,,,,,, and, and,,, a,,,,,, and,,,,, and and and,,,, and,,,, and and,,,,,,,,, and the, and,,,, and,,,,,,,,,,,,,,, and,,,,,,,,,,,, and,,, and,,,,,,,,,,,,,,,,,, and,,,,, and,,,,,,,, and,, and,,,,,,,,,, and, and,,,, and,,,, and and,,,,,,,,,, and,,, and,,,,,,, and and,,,,,, and,,,, and,,,,, and,,,,, and,,, and,, the and,,,,,,,,,,, and,,,,,,,,,,,,,,,,,,,,,, and and,,,, and,,,,,,,,,,,,,,,,,,,,,, and,,,,